In [3]:
# Minicons Installation
# Introduction can be found https://kanishka.xyz/post/minicons-running-large-scale-behavioral-analyses-on-transformer-lms/
# Tutorial and code can be found https://github.com/kanishkamisra/minicons/blob/master/examples/surprisals.md
#!pip install minicons

from minicons import scorer
import pandas as pd
import numpy as np
import json
import csv
import re
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import torch
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from transformers import AutoTokenizer, AutoModelForCausalLM

In [4]:
model_path = "ltg/gpt-bert-babylm-base"
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2TokenizerFast.from_pretrained(model_path)
lm_scorer = scorer.IncrementalLMScorer(model = model, tokenizer = tokenizer)

def calculate_surprisal(sentence):
    '''
    Takes in a sentence, and outputs surprisal values for each word.
    '''
    
    input_sentence = sentence
    # token_score() function of Minicons takes in several parameters
    # if surprisal = True, the output value is surprisal instead of log likelihood
    # if base_two = True, the log likelihood will be in base 2
    # see Minicons documentations for details
    # score tokens
    token_surprisals = lm_scorer.token_score(input_sentence, surprisal = True, base_two = True)[0]

    # filter out special tokens (like <pad>)
    special_tokens = set(tokenizer.all_special_tokens)
    filtered = [
        (token, score)
        for (token, score) in token_surprisals
        if token not in special_tokens
    ]

   # expand tokens that contain multiple words
    expanded = []
    #for token, score in token_surprisals:
    for token, score in filtered:
        token = token.strip('Ġ') # remove space marker
        token = token.lower()
        if token.count('Ġ') > 0:
            # multiple words inside
            words = token.split('Ġ')
            words = [word for word in words if word]  # remove empty strings
            for i, word in enumerate(words):
                expanded_token = word
                expanded.append((expanded_token, score / len(words)))  # split surprisal equally
        else:
            expanded.append((token, score))    
    #print(expanded)

    # use regex to split into words and punctuation
    words = re.findall(r'\w+|[^\w\s]', sentence.lower())
    results = []

    token_pointer = 0

    for word in words:
        accumulated = ''
        word_surprisal = 0.0

        while token_pointer < len(expanded):
            token, surprisal = expanded[token_pointer]
            accumulated += token
            word_surprisal += surprisal
            token_pointer += 1

            if accumulated == word:
                results.append((word, word_surprisal))
                break
        else:
            results.append((word, word_surprisal))

    return results


sentence = 'I know that your friend gave a baguette to Mary last weekend.'
calculate_surprisal(sentence)

[('i', 13.62649917602539),
 ('know', 14.585219383239746),
 ('that', 15.364814758300781),
 ('your', 14.07508373260498),
 ('friend', 14.361573219299316),
 ('gave', 11.86888599395752),
 ('a', 13.898443222045898),
 ('baguette', 45.41675281524658),
 ('to', 15.890382766723633),
 ('mary', 14.777327537536621),
 ('last', 15.184124946594238),
 ('weekend', 14.15632152557373),
 ('.', 14.957115173339844)]

In [12]:
model_path = "ltg/gpt-bert-babylm-base"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code = True)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code = True)
lm_scorer = scorer.IncrementalLMScorer(model = model, tokenizer = tokenizer)

def calculate_surprisal(sentence):
    
    # encode input text
    inputs = tokenizer(sentence, return_tensors = "pt")
    
    # forward pass to get logits
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs[0]
    
    # calculate log-probabilities in base 2
    log_probs = torch.nn.functional.log_softmax(logits, dim = -1) / torch.log(torch.tensor(2.0))
    
    # get token IDs and compute surprisal for each token in the sentence
    input_ids = inputs.input_ids[0]
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    
    token_surprisals = []
    for token, token_id in zip(tokens, input_ids):
        if token in tokenizer.all_special_tokens:
            continue  # skip special tokens
        # get the log-probability of the token
        log_prob = log_probs[0, tokens.index(token), token_id]  # use token index and id to find the log prob
        score = -log_prob.item()  # convert to scalar
        token_surprisals.append((token, score))
    print(token_surprisals)
        
    # expand tokens that contain multiple words
    expanded = []
    for token, score in token_surprisals:
    #for token, score in filtered:
        token = token.strip('Ġ') # remove space marker
        if token.count('Ġ') > 0:
            # multiple words inside
            words = token.split('Ġ')
            words = [word for word in words if word]  # remove empty strings
            for i, word in enumerate(words):
                expanded_token = word
                expanded.append((expanded_token, score / len(words)))  # split surprisal equally
        else:
            expanded.append((token, score))    

    # use regex to split into words and punctuation
    words = re.findall(r'\w+|[^\w\s]', sentence)
    results = []

    token_pointer = 0

    for word in words:
        accumulated = ''
        word_surprisal = 0.0

        while token_pointer < len(expanded):
            token, surprisal = expanded[token_pointer]
            accumulated += token
            word_surprisal += surprisal
            token_pointer += 1

            if accumulated == word:
                results.append((word, word_surprisal))
                break
        else:
            results.append((word, word_surprisal))

    return results

sentence = "I know that your friend gave a baguette to Mary last weekend."
surprisal_values = calculate_surprisal(sentence)
surprisal_values

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `2.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `2.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


[('ĠI', 10.14716911315918), ('Ġknow', 13.957598686218262), ('Ġthat', 7.678860187530518), ('Ġyour', 11.478602409362793), ('Ġfriend', 13.010708808898926), ('Ġgave', 16.35289764404297), ('Ġa', 9.544950485229492), ('Ġbag', 11.252456665039062), ('u', 17.371623992919922), ('ette', 16.360279083251953), ('Ġto', 11.33693790435791), ('ĠMary', 10.905172348022461), ('Ġlast', 14.300209999084473), ('Ġweekend', 17.16173553466797), ('.', 14.011561393737793)]


[('I', 10.14716911315918),
 ('know', 13.957598686218262),
 ('that', 7.678860187530518),
 ('your', 11.478602409362793),
 ('friend', 13.010708808898926),
 ('gave', 16.35289764404297),
 ('a', 9.544950485229492),
 ('baguette', 44.98435974121094),
 ('to', 11.33693790435791),
 ('Mary', 10.905172348022461),
 ('last', 14.300209999084473),
 ('weekend', 17.16173553466797),
 ('.', 14.011561393737793)]

In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

def calculate_surprisal(sentence, model_name='gpt2', base_two=False):
    """
    Computes the surprisal of each token in a sentence using a specified language model.

    Args:
        sentence (str): The input sentence for which to calculate surprisal.
        model_name (str): The name of the pre-trained model to use.
        base_two (bool): If True, returns surprisal in bits (log base 2); otherwise, in nats (log base e).

    Returns:
        list: A list of tuples containing each token and its corresponding surprisal.
    """
    # Initialize tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    model.eval()  # Set model to evaluation mode

    # Tokenize the input sentence
    inputs = tokenizer(sentence, return_tensors='pt')

    # Forward pass to get logits
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Compute log-probabilities
    log_probs = torch.nn.functional.log_softmax(logits, dim=-1)

    # Convert to base 2 if requested
    if base_two:
        log_probs /= torch.log(torch.tensor(2.0))

    # Extract token IDs and their corresponding tokens
    input_ids = inputs.input_ids[0]
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Calculate surprisal for each token
    token_surprisals = []
    for token, token_id in zip(tokens, input_ids):
        if token in tokenizer.all_special_tokens:
            continue  # Skip special tokens
        log_prob = log_probs[0, input_ids.tolist().index(token_id), token_id]
        surprisal = -log_prob.item()
        token_surprisals.append((token, surprisal))

    return token_surprisals


In [14]:
def calculate_sentence_surprisal(word_surprisals):
    '''Returns total surprisal and average surprisal per word.'''
    
    scores = [score for word, score in word_surprisals]
    total = sum(scores)
    avg = total / len(scores)
    return total, avg

"""
def sum_region_surprisal(row, region_list):
    '''Sums surprisals of a specified sentence region, extracting the relevant part from sentence_surprisals using index information.'''

    sentence_surprisals = row['surprisals']
    
    # retrieve all sentence column(from 'prefix' till 'end')
    all_columns = row.index.tolist()  # get the list of column names
    prefix_index = all_columns.index('prefix')
    eos_index = all_columns.index('end')
    
    # reconstruct the full sentence from 'prefix' to 'end' (exclude NaN values)
    full_sentence = ' '.join([str(row[col]) if pd.notna(row.get(col)) else '' for col in all_columns[prefix_index:eos_index + 1]])
    
    # use regex to split the full sentence into words & punctuation
    sentence_units = re.findall(r'\w+|[^\w\s]', full_sentence)
    
    # get the start and end indices for the region_list within the full sentence
    first_region_column = region_list[0]
    first_region_start_index = all_columns.index(first_region_column)
    
    last_region_column = region_list[-1]
    last_region_end_index = all_columns.index(last_region_column)
    
    # extract the relevant slice of sentence_surprisals that corresponds to the region_list
    # calculate the starting and ending index for the region_list part
    start_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:first_region_start_index]
        if pd.notna(row[col]) 
    ])
    end_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:last_region_end_index + 1]
        if pd.notna(row[col]) 
    ])

    # extract the slice of sentence_surprisals corresponding to the region_list part
    relevant_surprisals = sentence_surprisals[start_index:end_index]
    region_surprisal = sum([score for token, score in relevant_surprisals])
    
    return region_surprisal
"""

def sum_region_surprisal(row, region_list, priority_region = None, normalize = False):
    '''Sums surprisals of a specified sentence region with optional priority region handling.'''

    sentence_surprisals = row['surprisals']
    
    # retrieve all sentence column(from 'prefix' till 'end')
    all_columns = row.index.tolist()  # get the list of column names
    prefix_index = all_columns.index('prefix')
    eos_index = all_columns.index('end')
    
    # reconstruct the full sentence from 'prefix' to 'end' (exclude NaN values)
    full_sentence = ' '.join([str(row[col]) if pd.notna(row.get(col)) else '' for col in all_columns[prefix_index:eos_index + 1]])
    
    # use regex to split the full sentence into words & punctuation
    sentence_units = re.findall(r'\w+|[^\w\s]', full_sentence)
    
    if priority_region:
        # check if priority_region has non-zero surprisal
        if pd.notna(row[priority_region]):
            # extract text for the priority_region
            priority_region_index = all_columns.index(priority_region)
            
            start_index = sum([
                len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:priority_region_index]
                if pd.notna(row[col]) 
            ])
            end_index = sum([
                len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:priority_region_index + 1]
                if pd.notna(row[col]) 
            ])
            
            # calculate the surprisal for the priority region
            relevant_surprisals = sentence_surprisals[start_index:end_index]
            priority_surprisal = sum([score for token, score in relevant_surprisals])
    
            # return priority region's surprisal score if it is non-zero
            if priority_surprisal != 0:
                return priority_surprisal
    
    # if priority region is not provided or its surprisal is zero, calculate sum of region_list surprisals

    # get the start and end indices for the region_list within the full sentence
    first_region_column = region_list[0]
    first_region_start_index = all_columns.index(first_region_column)
    
    last_region_column = region_list[-1]
    last_region_end_index = all_columns.index(last_region_column)
    
    # extract the relevant slice of sentence_surprisals that corresponds to the region_list
    # calculate the starting and ending index for the region_list part
    start_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:first_region_start_index]
        if pd.notna(row[col]) 
    ])
    end_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:last_region_end_index + 1]
        if pd.notna(row[col]) 
    ])

    # extract the slice of sentence_surprisals corresponding to the region_list part
    relevant_surprisals = sentence_surprisals[start_index:end_index]
    region_surprisal = sum([score for token, score in relevant_surprisals])
    
    if normalize == False:
        return region_surprisal
    elif normalize == True:
        return region_surprisal / len(relevant_surprisals)

<>:24: SyntaxWarning: invalid escape sequence '\w'
<>:24: SyntaxWarning: invalid escape sequence '\w'
/var/folders/vv/sc6b0f596n7dy9mbxlcnn1180000gq/T/ipykernel_1375/2872888706.py:24: SyntaxWarning: invalid escape sequence '\w'
  sentence_units = re.findall(r'\w+|[^\w\s]', full_sentence)


In [8]:
def split_ends(ends):
    '''Splits off sentence-final punctuation.'''
    
    match = re.match(r'^(.*?)([.!?])$', ends.strip())
    if match:
        return pd.Series([match.group(1), match.group(2)])
    else:
        return pd.Series([ends, ''])  # no end punctuation
    
def encode_wh_licensor(df):
    '''
    Adds a numeric column to the DataFrame:
    - 1 if licensor is a wh-licensor
    - -1 otherwise (licensor is "that" or absent)
    '''
    df['wh_numeric'] = df['filler'].apply(lambda x: 1 if x > 0 else -1)
    return df

def merge_sentence(row, syntactic_parts):
    '''Merges sentence columns back into full sentence.'''
    
    parts = [row[part] for part in syntactic_parts]         
    # filter out NaN or empty parts
    non_empty_parts = [str(part) for part in parts if pd.notna(part) and str(part).strip() != '']
    # join with spaces and add end punctuation
    sentence = ' '.join(non_empty_parts) + (row['end'] if pd.notna(row['end']) else '')
    return sentence

#### Statistical Analysis: Mixed-Effects Linear Regression Model

In [9]:
from IPython.display import display
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning

def print_summary(result):
    '''
    Prints the descriptive header from a statsmodels MixedLMResults summary,
    and displays both the fixed-effects and random-effects components as pandas DataFrames.
    '''
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", ConvergenceWarning)
        
        summary_str = str(result.summary())
        lines = summary_str.split('\n')

        # PART 1: HEADER
        print("\n=== MODEL SUMMARY ===")
        for line in lines:
            if 'Coef.' in line:
                break  # stop when reaching the coefficient table
            pairs = re.findall(r'(\S[^:]*):\s+([^\s][^:]*?)(?=\s{2,}|$)', line)
            for key, value in pairs:
                print(f"{key.strip()}: {value.strip()}")

        # PART 2a: FIXED EFFECTS TABLE
        fixed_table = []
        in_fixed_table = False
        random_start_idx = None

        for i, line in enumerate(lines):
            if 'Coef.' in line:
                in_fixed_table = True
                continue
            if in_fixed_table:
                if not line.strip() or set(line.strip()) in [{'='}, {'-'}]:
                    continue  # skip empty/separator lines
                parts = line.strip().split()
                if len(parts) >= 7:
                    param = parts[0]
                    row = parts[1:7]
                    fixed_table.append([param] + row)
                else:
                    random_start_idx = i
                    break

        if fixed_table:
            df_fixed = pd.DataFrame(fixed_table, columns = [
                'Parameter', 'Coef.', 'Std.Err.', 'z', 'P>|z|', '[0.025', '0.975]'
            ])
            print("\n=== FIXED EFFECTS COEFFICIENTS ===")
            display(df_fixed)
        else:
            print("\nNo fixed-effects data found.")

        # PART 2b: RANDOM EFFECTS / VARIANCE COMPONENTS TABLE
        random_table = []
        if random_start_idx:
            for line in lines[random_start_idx:]:
                if not line.strip() or set(line.strip()) in [{'='}, {'-'}]:
                    continue
                parts = line.strip().split()
                # Look for last token ending in Var or Cov
                varcov_idx = None
                for j in reversed(range(len(parts))):
                    if parts[j].endswith('Var') or parts[j].endswith('Cov'):
                        varcov_idx = j
                        break
                if varcov_idx is not None:
                    param = ' '.join(parts[:varcov_idx + 1])
                    numeric_parts = parts[varcov_idx + 1:]
                    if len(numeric_parts) == 2:
                        val1, val2 = numeric_parts
                    elif len(numeric_parts) == 1:
                        val1, val2 = numeric_parts[0], ''
                    else:
                        val1, val2 = '', ''
                    random_table.append([param, val1, val2])

        if random_table:
            df_random = pd.DataFrame(random_table, columns = ['Parameter', 'Coef.', 'Std.Err.'])
            print("\n=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===")
            display(df_random)
        else:
            print("\nNo random-effects data found.")


In [10]:
# columns: item_id (indicates sentence set), wh_licensor (0/1), gap (0/1), island_type, surprisal

warnings.simplefilter("ignore", ConvergenceWarning)

def mixed_effects_linear_regression(df, iv_list, surprisal, label):
    '''
    Fits mixed-effects model and extracts wh-licensing interaction.
    '''
    '''    
    for field in iv_list:
        if field == 'wh_numeric' or 'gap_numeric' or 'gap_distance_length':
            pass
        else:
            df[field] = df[field].astype('category')
    '''

    interaction_terms = ' * '.join(iv_list)
    random_effects = ' + '.join(iv_list)

    model = smf.mixedlm(
        f"{surprisal} ~ {interaction_terms}",
        df,
        groups = df["item_id"],
        #re_formula = f"~{random_effects}"
        re_formula = "1"
        )

    result = model.fit()
    #interaction_coef = result.params.get('wh_licensor[T.1]:gap[T.1]', None)

    print(f"\n=== {label.upper()} ===")
    #print(result.summary())
    print_summary(result)

    return result

#interaction = mixed_effects_linear_regression(df, "construction_type") # label name to be changed according to construction type

#### Gap Distance - Categorical/Continuous

In [11]:
sentence_df = pd.read_csv('test_sentences/Gap Distance.csv')
sentence_df['gap_distance'] = sentence_df['gap_distance'].fillna('null')
#sentence_df['gap_distance'].dropna(inplace = True)
#sentence_df = sentence_df[sentence_df['gap_distance'].replace(['nan', 'NaN'], np.nan).notna()]

FileNotFoundError: [Errno 2] No such file or directory: 'test_sentences/Gap Distance.csv'

In [ ]:
sentence_df[['temp_mod', 'end']] = sentence_df['temp_mod'].apply(split_ends)
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'licensor', 'subj', 'modifier', 'verb', 'object', 'prep', 'goal', 'temp_mod']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence(x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['gap_distance_length'] = sentence_df['modifier'].apply(lambda x: len(x) if pd.notna(x) else 0)
sentence_df['gap_distance'] = pd.Categorical(sentence_df['gap_distance'], categories = ['null', 'short', 'medium', 'long'], ordered = True)

Gap Position == Object

In [ ]:
# modifier: short_mod, med_mod, long_mod

In [ ]:
object_df = sentence_df.copy()
#object_df['region_surprisal'] = object_df.apply(lambda x: sum_region_surprisal(x, ['prep', 'goal']), axis = 1)
object_df['region_surprisal'] = object_df.apply(lambda x: sum_region_surprisal(x, ['prep', 'goal'], 'object'), axis = 1)

# filter for object gap
objectgap_df = object_df[object_df['gap_position'] == 'DO']

In [ ]:
# categorical
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance']
mixed_effects_linear_regression(objectgap_df, iv_list, 'region_surprisal', 'gap distance: gap position at object')


=== GAP DISTANCE: GAP POSITION AT OBJECT ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 125.3361
Min. group size: 16
Log-Likelihood: -1221.8000
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,27.646,2.185,12.651,0.000,23.363,31.929
1,gap_distance[T.short],-2.534,1.770,-1.432,0.152,-6.004,0.935
2,gap_distance[T.medium],-3.426,1.770,-1.935,0.053,-6.895,0.043
3,gap_distance[T.long],-9.448,1.770,-5.338,0.000,-12.918,-5.979
4,wh_numeric,0.231,1.252,0.185,0.853,-2.222,2.685
5,wh_numeric:gap_distance[T.short],-0.158,1.770,-0.089,0.929,-3.628,3.311
6,wh_numeric:gap_distance[T.medium],-0.207,1.770,-0.117,0.907,-3.677,3.262
7,wh_numeric:gap_distance[T.long],-0.189,1.770,-0.107,0.915,-3.659,3.280
8,gap_numeric,3.107,1.252,2.483,0.013,0.654,5.561
9,gap_numeric:gap_distance[T.short],0.029,1.770,0.016,0.987,-3.441,3.498



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,64.176,2.155


In [ ]:
# continuous
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance_length']
mixed_effects_linear_regression(objectgap_df, iv_list, 'region_surprisal', 'gap distance: gap position at object')


=== GAP DISTANCE: GAP POSITION AT OBJECT ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 122.1466
Min. group size: 16
Log-Likelihood: -1249.6896
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,28.176,2.008,14.033,0.000,24.241,32.111
1,wh_numeric,0.169,0.983,0.172,0.863,-1.757,2.095
2,gap_numeric,3.151,0.983,3.206,0.001,1.225,5.076
3,wh_numeric:gap_numeric,0.109,0.983,0.111,0.911,-1.816,2.035
4,gap_distance_length,-0.124,0.022,-5.692,0.000,-0.166,-0.081
5,wh_numeric:gap_distance_length,-0.002,0.022,-0.100,0.920,-0.044,0.040
6,gap_numeric:gap_distance_length,-0.022,0.022,-1.021,0.307,-0.064,0.020
7,wh_numeric:gap_numeric:gap_distance_length,-0.002,0.022,-0.094,0.925,-0.044,0.040



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,61.143,2.084


Gap Postion == PP/Goal

In [ ]:
pp_df = sentence_df.copy()
#pp_df['region_surprisal'] = pp_df.apply(lambda x: sum_region_surprisal(x, ['temp_mod']), axis = 1)
pp_df['region_surprisal'] = pp_df.apply(lambda x: sum_region_surprisal(x, ['temp_mod'], 'goal'), axis = 1)

# filter for PP gap
ppgap_df = pp_df[pp_df['gap_position'] == 'IO']

In [ ]:
# categorical
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance']
mixed_effects_linear_regression(ppgap_df, iv_list, 'region_surprisal', 'gap distance: gap position at pp/goal')


=== GAP DISTANCE: GAP POSITION AT PP/GOAL ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 126.7513
Min. group size: 16
Log-Likelihood: -1216.9029
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,24.233,1.734,13.979,0.000,20.835,27.631
1,gap_distance[T.short],-2.796,1.780,-1.570,0.116,-6.285,0.693
2,gap_distance[T.medium],-2.129,1.780,-1.196,0.232,-5.618,1.360
3,gap_distance[T.long],-8.136,1.780,-4.570,0.000,-11.625,-4.647
4,wh_numeric,-0.109,1.259,-0.087,0.931,-2.576,2.358
5,wh_numeric:gap_distance[T.short],-0.068,1.780,-0.038,0.969,-3.557,3.421
6,wh_numeric:gap_distance[T.medium],-0.008,1.780,-0.005,0.996,-3.497,3.481
7,wh_numeric:gap_distance[T.long],0.040,1.780,0.023,0.982,-3.449,3.529
8,gap_numeric,1.098,1.259,0.872,0.383,-1.369,3.565
9,gap_numeric:gap_distance[T.short],-1.056,1.780,-0.593,0.553,-4.545,2.433



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,28.417,1.082


In [ ]:
# continuous
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance_length']
mixed_effects_linear_regression(ppgap_df, iv_list, 'region_surprisal', 'gap distance: gap position at pp/goal')


=== GAP DISTANCE: GAP POSITION AT PP/GOAL ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 124.6591
Min. group size: 16
Log-Likelihood: -1246.2858
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,24.596,1.536,16.013,0.000,21.585,27.606
1,wh_numeric,-0.150,0.993,-0.151,0.880,-2.096,1.795
2,gap_numeric,0.785,0.993,0.791,0.429,-1.160,2.731
3,wh_numeric:gap_numeric,-0.236,0.993,-0.238,0.812,-2.182,1.710
4,gap_distance_length,-0.102,0.022,-4.669,0.000,-0.145,-0.059
5,wh_numeric:gap_distance_length,0.001,0.022,0.042,0.967,-0.042,0.044
6,gap_numeric:gap_distance_length,0.009,0.022,0.410,0.682,-0.034,0.052
7,wh_numeric:gap_numeric:gap_distance_length,0.002,0.022,0.103,0.918,-0.040,0.045



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,27.320,1.053


Embedded Clause

In [ ]:
embed = ['subj', 'modifier', 'verb', 'object', 'prep', 'goal', 'temp_mod', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

Embedded Clause - Object Gap Position

In [ ]:
# filter for object gap
embed_objectgap_df = sentence_df[sentence_df['gap_position'] == 'DO']

In [ ]:
# categorical
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance']
mixed_effects_linear_regression(embed_objectgap_df, iv_list, 'embed_surprisal', 'gap distance: embedded clause object gap position')


=== GAP DISTANCE: EMBEDDED CLAUSE OBJECT GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.3723
Min. group size: 16
Log-Likelihood: -345.8430
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,6.971,0.173,40.399,0.000,6.633,7.309
1,gap_distance[T.short],0.456,0.096,4.724,0.000,0.267,0.645
2,gap_distance[T.medium],1.018,0.096,10.554,0.000,0.829,1.207
3,gap_distance[T.long],1.698,0.096,17.605,0.000,1.509,1.888
4,wh_numeric,0.080,0.068,1.177,0.239,-0.053,0.214
5,wh_numeric:gap_distance[T.short],-0.046,0.096,-0.474,0.635,-0.235,0.143
6,wh_numeric:gap_distance[T.medium],-0.046,0.096,-0.477,0.633,-0.235,0.143
7,wh_numeric:gap_distance[T.long],-0.063,0.096,-0.658,0.511,-0.253,0.126
8,gap_numeric,-0.179,0.068,-2.621,0.009,-0.312,-0.045
9,gap_numeric:gap_distance[T.short],0.048,0.096,0.495,0.621,-0.141,0.237



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.502,0.289


In [ ]:
# continuous
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance_length']
mixed_effects_linear_regression(embed_objectgap_df, iv_list, 'embed_surprisal', 'gap distance: embedded clause object gap position')


=== GAP DISTANCE: EMBEDDED CLAUSE OBJECT GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.3398
Min. group size: 16
Log-Likelihood: -340.7323
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,6.952,0.163,42.719,0.000,6.633,7.271
1,wh_numeric,0.066,0.052,1.282,0.200,-0.035,0.168
2,gap_numeric,-0.178,0.052,-3.438,0.001,-0.280,-0.077
3,wh_numeric:gap_numeric,0.040,0.052,0.778,0.436,-0.061,0.142
4,gap_distance_length,0.023,0.001,20.000,0.000,0.021,0.025
5,wh_numeric:gap_distance_length,-0.001,0.001,-0.620,0.535,-0.003,0.002
6,gap_numeric:gap_distance_length,0.002,0.001,1.591,0.112,-0.000,0.004
7,wh_numeric:gap_numeric:gap_distance_length,-0.001,0.001,-0.517,0.605,-0.003,0.002



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.475,0.285


Embedded Clause - PP/Goal Gap Position

In [ ]:
# filter for PP gap
embed_ppgap_df = sentence_df[sentence_df['gap_position'] == 'IO']

In [ ]:
# categorical
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance']
mixed_effects_linear_regression(embed_ppgap_df, iv_list, 'embed_surprisal', 'gap distance: embedded clause pp/goal gap position')


=== GAP DISTANCE: EMBEDDED CLAUSE PP/GOAL GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.3628
Min. group size: 16
Log-Likelihood: -344.0243
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,6.778,0.188,36.055,0.000,6.410,7.146
1,gap_distance[T.short],0.502,0.095,5.269,0.000,0.315,0.688
2,gap_distance[T.medium],1.110,0.095,11.657,0.000,0.924,1.297
3,gap_distance[T.long],1.805,0.095,18.957,0.000,1.619,1.992
4,wh_numeric,-0.067,0.067,-0.988,0.323,-0.199,0.065
5,wh_numeric:gap_distance[T.short],0.009,0.095,0.092,0.927,-0.178,0.195
6,wh_numeric:gap_distance[T.medium],0.024,0.095,0.257,0.797,-0.162,0.211
7,wh_numeric:gap_distance[T.long],0.022,0.095,0.228,0.820,-0.165,0.208
8,gap_numeric,-0.300,0.067,-4.461,0.000,-0.432,-0.168
9,gap_numeric:gap_distance[T.short],0.092,0.095,0.967,0.334,-0.095,0.279



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.616,0.355


In [ ]:
# continuous
iv_list = ['wh_numeric', 'gap_numeric', 'gap_distance_length']
mixed_effects_linear_regression(embed_ppgap_df, iv_list, 'embed_surprisal', 'gap distance: embedded clause pp/goal gap position')


=== GAP DISTANCE: EMBEDDED CLAUSE PP/GOAL GAP POSITION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.3280
Min. group size: 16
Log-Likelihood: -337.5753
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,6.769,0.180,37.686,0.000,6.417,7.121
1,wh_numeric,-0.062,0.051,-1.218,0.223,-0.162,0.038
2,gap_numeric,-0.285,0.051,-5.594,0.000,-0.385,-0.185
3,wh_numeric:gap_numeric,-0.017,0.051,-0.329,0.742,-0.117,0.083
4,gap_distance_length,0.024,0.001,21.628,0.000,0.022,0.027
5,wh_numeric:gap_distance_length,0.000,0.001,0.232,0.817,-0.002,0.002
6,gap_numeric:gap_distance_length,0.003,0.001,2.627,0.009,0.001,0.005
7,wh_numeric:gap_numeric:gap_distance_length,0.000,0.001,0.179,0.858,-0.002,0.002



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.593,0.359


#### Double Gaps

In [ ]:
sentence_df = pd.read_csv('test_sentences/Double Gaps.csv')

In [ ]:
sentence_df[['modifier', 'end']] = sentence_df['modifier'].apply(split_ends)
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'licensor', 'subj', 'verb', 'object', 'modifier']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence(x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['gap_count'] = sentence_df['subject_gap'] + sentence_df['object_gap']

In [ ]:
sentence_df['gap_count'] = pd.Categorical(sentence_df['gap_count'], categories = [1, 0, 2], ordered = True)

Embedded Clause

In [ ]:
embed = ['subj', 'object', 'verb', 'modifier', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

In [ ]:
iv_list = ['wh_numeric', 'gap_count']
result = mixed_effects_linear_regression(sentence_df, iv_list, 'embed_surprisal', 'double gap embed')
result


=== DOUBLE GAP EMBED ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 0.2654
Min. group size: 8
Log-Likelihood: -154.9999
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,6.014,0.172,34.884,0.000,5.676,6.351
1,gap_count[T.0],0.493,0.100,4.942,0.000,0.298,0.689
2,gap_count[T.2],-0.784,0.100,-7.858,0.000,-0.979,-0.588
3,wh_numeric,0.085,0.058,1.475,0.140,-0.028,0.198
4,wh_numeric:gap_count[T.0],-0.009,0.100,-0.094,0.925,-0.205,0.186
5,wh_numeric:gap_count[T.2],-0.020,0.100,-0.205,0.837,-0.216,0.175



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.528,0.377


#### Wh-Islands

In [ ]:
sentence_df = pd.read_csv('test_sentences/Wh-Islands.csv')
sentence_df['comp_type'] = sentence_df['comp_type'].fillna('null')

In [ ]:
sentence_df[['cont', 'end']] = sentence_df['cont'].apply(split_ends)
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'filler_word', 'subj1', 'verb1', 'comp', 'subj2', 'embed_verb', 'obj', 'cont']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence (x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['comp_type'] = pd.Categorical(sentence_df['comp_type'], categories = ['null', 'that', 'whether'], ordered = True)

Post-Gap Region - Continuation

In [ ]:
sentence_df['region_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, ['cont'], 'obj'), axis = 1)

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'comp_type'] # island_type = comp_type
mixed_effects_linear_regression(sentence_df, iv_list, 'region_surprisal', 'wh-islands continuation')


=== WH-ISLANDS CONTINUATION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 30.2172
Min. group size: 12
Log-Likelihood: -761.6864
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,24.603,1.310,18.774,0.000,22.034,27.172
1,comp_type[T.that],0.176,0.869,0.203,0.839,-1.527,1.880
2,comp_type[T.whether],0.269,0.869,0.310,0.757,-1.434,1.973
3,wh_numeric,-0.045,0.615,-0.074,0.941,-1.250,1.159
4,wh_numeric:comp_type[T.that],0.155,0.869,0.178,0.858,-1.548,1.859
5,wh_numeric:comp_type[T.whether],0.189,0.869,0.218,0.828,-1.514,1.893
6,gap_numeric,5.306,0.615,8.633,0.000,4.101,6.510
7,gap_numeric:comp_type[T.that],0.047,0.869,0.055,0.956,-1.656,1.751
8,gap_numeric:comp_type[T.whether],-0.125,0.869,-0.144,0.886,-1.828,1.579
9,wh_numeric:gap_numeric,0.030,0.615,0.049,0.961,-1.175,1.235



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,26.793,1.807


Embedded Clause

In [ ]:
embed = ['subj1', 'verb1', 'comp', 'subj2', 'embed_verb', 'obj', 'cont', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'comp_type'] # island_type = comp_type
mixed_effects_linear_regression(sentence_df, iv_list, 'embed_surprisal', 'wh-islands embed')


=== WH-ISLANDS EMBED ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 0.0428
Min. group size: 12
Log-Likelihood: -52.8088
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.133,0.355,22.911,0.000,7.437,8.829
1,comp_type[T.that],-0.128,0.033,-3.925,0.000,-0.192,-0.064
2,comp_type[T.whether],0.057,0.033,1.729,0.084,-0.008,0.121
3,wh_numeric,-0.228,0.023,-9.871,0.000,-0.274,-0.183
4,wh_numeric:comp_type[T.that],0.028,0.033,0.850,0.396,-0.036,0.092
5,wh_numeric:comp_type[T.whether],0.047,0.033,1.438,0.150,-0.017,0.111
6,gap_numeric,-0.010,0.023,-0.452,0.651,-0.056,0.035
7,gap_numeric:comp_type[T.that],-0.020,0.033,-0.612,0.541,-0.084,0.044
8,gap_numeric:comp_type[T.whether],-0.019,0.033,-0.596,0.551,-0.084,0.045
9,wh_numeric:gap_numeric,-0.014,0.023,-0.595,0.552,-0.059,0.032



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,2.510,4.115


#### Adjunct Islands 

In [ ]:
sentence_df = pd.read_csv('test_sentences/Adjunct Islands.csv')
sentence_df['adjunct_type'] = sentence_df['adjunct_type'].fillna('null')

In [ ]:
sentence_df[['cont', 'end']] = sentence_df['cont'].apply(split_ends)
sentence_df['adjunct_front_trigger'] = np.nan
sentence_df['adjunct_front_trigger'] = sentence_df['adjunct_trigger'].where(sentence_df['adjunct_type'] == 'front', sentence_df['adjunct_front_trigger'])
sentence_df['adjunct_trigger'] = sentence_df['adjunct_trigger'].where(sentence_df['adjunct_type'] != 'front', np.nan)
sentence_df.rename(columns = {'adjunct_trigger': 'adjunct_back_trigger'}, inplace = True)
sentence_df = sentence_df[[col for col in sentence_df.columns if col != 'end'] + ['end']]

In [ ]:
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'filler_word', 'adjunct_back_trigger', 'prep', 'subj', 'mod', 'verb', 'obj', 'cont', 'adjunct_front_trigger']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence(x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['adjunct_type'] = pd.Categorical(sentence_df['adjunct_type'], categories = ['null', 'back', 'front'], ordered = True)

Post-Gap Region - Continuation

In [ ]:
sentence_df['region_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, ['cont'], 'obj'), axis = 1)

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'adjunct_type'] # island_type = adjunct_type
mixed_effects_linear_regression(sentence_df, iv_list, 'region_surprisal', 'adjunct islands continuation')


=== ADJUNCT ISLANDS CONTINUATION ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 118.5236
Min. group size: 12
Log-Likelihood: -916.6340
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,33.063,2.498,13.235,0.000,28.167,37.959
1,adjunct_type[T.back],1.018,1.721,0.591,0.554,-2.356,4.392
2,adjunct_type[T.front],0.145,1.721,0.084,0.933,-3.229,3.518
3,wh_numeric,0.145,1.217,0.119,0.905,-2.241,2.530
4,wh_numeric:adjunct_type[T.back],-0.132,1.721,-0.077,0.939,-3.506,3.242
5,wh_numeric:adjunct_type[T.front],-0.386,1.721,-0.224,0.823,-3.760,2.988
6,gap_numeric,2.042,1.217,1.678,0.093,-0.343,4.428
7,gap_numeric:adjunct_type[T.back],0.270,1.721,0.157,0.875,-3.104,3.644
8,gap_numeric:adjunct_type[T.front],0.154,1.721,0.090,0.929,-3.220,3.528
9,wh_numeric:gap_numeric,-0.182,1.217,-0.149,0.881,-2.567,2.204



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,95.183,3.270


Embedded Clause

In [ ]:
embed = ['adjunct_back_trigger', 'prep', 'subj', 'mod', 'verb', 'obj', 'cont', 'adjunct_front_trigger', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'adjunct_type'] # island_type = adjunct_type
mixed_effects_linear_regression(sentence_df, iv_list, 'embed_surprisal', 'adjunct islands embed')


=== ADJUNCT ISLANDS EMBED ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 0.2589
Min. group size: 12
Log-Likelihood: -240.0621
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.399,0.342,24.590,0.000,7.730,9.069
1,adjunct_type[T.back],0.648,0.080,8.057,0.000,0.491,0.806
2,adjunct_type[T.front],0.406,0.080,5.050,0.000,0.249,0.564
3,wh_numeric,0.138,0.057,2.430,0.015,0.027,0.250
4,wh_numeric:adjunct_type[T.back],-0.111,0.080,-1.383,0.167,-0.269,0.046
5,wh_numeric:adjunct_type[T.front],-0.195,0.080,-2.422,0.015,-0.353,-0.037
6,gap_numeric,-0.302,0.057,-5.302,0.000,-0.413,-0.190
7,gap_numeric:adjunct_type[T.back],0.093,0.080,1.160,0.246,-0.064,0.251
8,gap_numeric:adjunct_type[T.front],0.096,0.080,1.196,0.232,-0.061,0.254
9,wh_numeric:gap_numeric,0.014,0.057,0.240,0.811,-0.098,0.125



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,2.269,1.525


#### Complex NP Islands

In [ ]:
sentence_df = pd.read_csv('test_sentences/Complex NP Islands.csv')

In [ ]:
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
sentence_df['sentence'] = sentence_df.apply(merge_sentence, axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df

Subject Condition

In [ ]:
subject_df = sentence_df.copy()

In [ ]:
subject_df['region_surprisal'] = subject_df.apply(lambda x: sum_region_surprisal(x, ['subj_setup']), axis = 1)
subject_df

In [ ]:
# filter for subject condition
subject_df = subject_df[subject_df['subj_obj'] == 'subject']

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'island_type']
mixed_effects_linear_regression(subject_df, iv_list, 'region_surprisal', 'complex np islands: subject condition')

Object Condition

In [ ]:
object_df = sentence_df.copy()

In [ ]:
object_df['region_surprisal'] = object_df.apply(lambda x: sum_region_surprisal(x, ['end']), axis = 1)
object_df

In [ ]:
# filter for object gap
object_df = object_df[object_df['subj_obj'] == 'object']

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'island_type']
mixed_effects_linear_regression(object_df, iv_list, 'region_surprisal', 'complex np islands: object condition')

Embedded Clause

In [ ]:
embed = ['subj', 'that_rc', 'what_rc', 'rc_np', 'prep', 'prep_np', 'subj_setup', 'obj_setup', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)
sentence_df

Embedded Clause - Subject Condition

In [ ]:
# filter for subject condition
subject_df = sentence_df[sentence_df['subj_obj'] == 'subject']

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'island_type']
mixed_effects_linear_regression(subject_df, iv_list, 'embed_surprisal', 'complex np islands: embedded clause subject condition')

Embedded Clause - Object Condition

In [ ]:
# filter for object gap
object_df = sentence_df[sentence_df['subj_obj'] == 'object']

In [ ]:
iv_list = ['wh_numeric', 'gap_numeric', 'island_type']
mixed_effects_linear_regression(object_df, iv_list, 'embed_surprisal', 'complex np islands: embedded clause object condition')